In [10]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import wikipediaapi

import requests, json, re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook


In [7]:
wiki = wikipediaapi.Wikipedia('en')

In [8]:
freshmen_1118 = []

def get_debut_albums():
    
    for i in tqdm_notebook(range(1, 8+1)):
        result = wiki.page("Category:201"+str(i)+" debut albums")
        freshmen_1118.extend(list(result.categorymembers))
    
    print(len(freshmen_1118))

In [9]:
get_debut_albums()

HBox(children=(IntProgress(value=0, max=8), HTML(value='')))


2007


In [14]:
def get_freshmen_data(list):
    
    df = pd.DataFrame(columns=['artist', 'album', 'genre', 'single_count'])
    
    for albums in tqdm_notebook(list):
        url = "https://en.wikipedia.org/wiki/" + str(albums)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        tmp_tag = soup.find(id='content')
        
        
        try:
            artist = tmp_tag.find(class_="contributor").get_text()
        except AttributeError:
            artist = "N/A"
        
        
        try:
            album = tmp_tag.find(class_="summary").get_text()
        except AttributeError:
            album = "N/A"
        
        genre_ls = []
        try:
            for i in range(len(tmp_tag.select('.category a[href*="/wiki/"]'))):
                genre = tmp_tag.select('.category a[href*="/wiki/"]')[i].get_text()
                genre_ls.append(genre)
        except AttributeError:
            genre_ls = []
        genre_ls = ", ".join(genre_ls)
        
        try:
            single_count = len(tmp_tag.select("tr td div ol li"))
        except AttributeError:
            single_count = 0
        

        
        
        data = {"artist" : artist,
            "album" : album,
            "genre" : genre_ls,
            "single_count" : single_count, }

        df.loc[len(df)] = data
    
    return df

In [16]:
df = get_freshmen_data(freshmen_1118)

HBox(children=(IntProgress(value=0, max=2007), HTML(value='')))

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [32]:
print(df.shape)
df.tail(20)

(2007, 4)


,artist,album,genre,single_count
1987,Terror Universal,Make Them Bleed,Heavy metal,0
1988,NCT,NCT 2018 Empathy,"K-pop, hip hop, future bass, R&B, electropop",6
1989,Calum Scott,Only Human,,3
1990,Kasbo,Places We Don't Know,"Electronic, future bass, house",6
1991,Mike Shinoda,Post Traumatic,,1
1992,Maxo Kream,Punken,Trap,0
1993,Rich Homie Quan,Rich as in Spirit,Hip hop,2
1994,Shame,Songs of Praise,"Post-punk, indie rock",0
1995,Anne-Marie,Speak Your Mind,,5
1996,Ängie,Suicidal Since 1995,,2


### Examine Scraping Errors

In [52]:
df2 = df[df['album'].isin(['N/A']) == False]

In [53]:
# Row 1735 happens to be K-pop group "Nine Muses"
# Not relevant to the data purpose, thus drop

df2[df2['artist'].isin(['N/A'])]

,artist,album,genre,single_count
319,N/A,Talk About Body,"Electropop, Political music",3
1735,N/A,"""Muses Diary""",,0


In [54]:
df2.drop([1735], inplace=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [55]:
# Fill in artist info for row 1735 (https://en.wikipedia.org/wiki/Talk_About_Body)

df2[df2['artist'].isin(['N/A'])]

,artist,album,genre,single_count
319,N/A,Talk About Body,"Electropop, Political music",3


In [56]:
df2['artist'][319] = 'MEN'

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


### Drop duplicate entries

In [57]:
# Drop duplicate entries. 
# Keep the earlier debut album, drop the latter ones.

df2.drop_duplicates(subset='artist', keep='first', inplace=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [72]:
df2.to_csv("../data/debut_album_1118.csv", index=False)

### Fill in missing genres my manual imputation

In [73]:
df2 = pd.read_csv("../data/debut_album_1118.csv")

In [74]:
df2.head()

,artist,album,genre,single_count
0,2Cellos,2Cellos,"Cello rock, classical crossover",3
1,Deep Dark Robot,8 Songs About a Girl,"Post-grunge, punk blues",0
2,The Young Professionals,"9am to 5pm, 5pm to Whenever\r(09:00 to 17:00, ...",Electronic,0
3,Flash Bang Grenada,10 Haters,Alternative hip hop,0
4,Her Majesty & the Wolves,111,"Dance, pop",3


### Save to csv file

In [60]:
df2.to_csv("../data/debut_album_1118.csv", index=False)

### Load csv file to double check

In [7]:
df = pd.read_csv("../data/debut_album_1118.csv")

In [8]:
print(df.shape)
df.head()

(1949, 4)


,artist,album,genre,single_count
0,2Cellos,2Cellos,"Cello rock, classical crossover",3
1,Deep Dark Robot,8 Songs About a Girl,"Post-grunge, punk blues",0
2,The Young Professionals,"9am to 5pm, 5pm to Whenever\r(09:00 to 17:00, ...",Electronic,0
3,Flash Bang Grenada,10 Haters,Alternative hip hop,0
4,Her Majesty & the Wolves,111,"Dance, pop",3


### Upload to MySQL database

In [17]:
import sqlalchemy, pickle
from sqlalchemy import create_engine

In [21]:
pw = pickle.load(open("mysql_pw.pickle", "rb"))

In [22]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + pw + "@52.78.44.120/project_rookie")

In [23]:
df.to_sql(name="debut_album", con=engine, if_exists='replace')